<a href="https://colab.research.google.com/github/edgekrtl/100-days-of-code/blob/master/CUDA_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA


In [4]:
%%file example.c
#include <stdio.h>

__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1000;
    int *a, *b, *c;  // Host (CPU) belleği için vektörler
    int *d_a, *d_b, *d_c;  // Device (GPU) belleği için vektörler

    // Bellek ayırma
    a = (int*)malloc(n * sizeof(int));
    b = (int*)malloc(n * sizeof(int));
    c = (int*)malloc(n * sizeof(int));

    // Host (CPU) belleğinde vektörlerin değerlerini ata
    for (int i = 0; i < n; i++) {
        a[i] = i;
        b[i] = i;
    }

    // Device (GPU) belleği için bellek ayırma
    cudaMalloc(&d_a, n * sizeof(int));
    cudaMalloc(&d_b, n * sizeof(int));
    cudaMalloc(&d_c, n * sizeof(int));

    // Host'tan Device'a veri kopyalama
    cudaMemcpy(d_a, a, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, n * sizeof(int), cudaMemcpyHostToDevice);

    // CUDA işlemi için blok ve iş parçacığı konfigürasyonu
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;

    // CUDA işlemini başlatma
    vectorAdd<<<numBlocks, blockSize>>>(d_a, d_b, d_c, n);

    // Sonucu Device'tan Host'a kopyalama
    cudaMemcpy(c, d_c, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Sonucu ekrana yazdırma
    for (int i = 0; i < n; i++) {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    // Bellekleri serbest bırakma
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Overwriting example.c
